# What is the relationship between the Gross Domestic Product Per Capita and the performance of their soccer teams in international matches?

## Introduction

- Find bias
- Regression, based on country's GDP and performance
- Dataset

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(rvest)
library(stringr)
library(readxl)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

## Data Sources

- https://www.imf.org/external/datamapper/datasets/WEO
- https://www.rsssf.org/tablesw/wcq-records.html

In [9]:
# Loading all data needed for this project
country_gdp <- read_excel(path = "data/country_gdp.xls")
country_gdp
soccer_data <- read_csv("data/SoccerDataWorldCup.csv")
soccer_data

"GDP per capita, current prices (U.S. dollars per capita)",1980,1981,1982,1983,1984,1985,1986,1987,1988,⋯,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Afghanistan,no data,no data,no data,no data,no data,no data,no data,no data,no data,⋯,582.32299999999998,586.20399999999995,611.26800000000003,no data,no data,no data,no data,no data,no data,no data
Albania,728.35900000000004,817.73400000000004,824.54200000000003,815.529,788.33199999999999,788.80100000000004,855.72400000000005,832.245,805.04600000000005,⋯,5257.7139999999999,5345.058,5268.2389999999996,6373.3000000000002,6369.009,6591.6819999999998,6968.6940000000004,7470.0820000000003,7958.0159999999996,8469.5069999999996
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
World,2862.3330000000001,2876.2199999999998,2772.2150000000001,2797.085,2846.6729999999998,2920.23,3376.8209999999999,3820.7339999999999,4222.9939999999997,⋯,11489.302,11558.645,11155.513999999999,12616.281000000001,13396.102000000001,13923.716,14522.870999999999,15149.837,15810.007,16490.797999999999
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
"©IMF, 2022",NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Rows: 212 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): Country, WC1, GD, P2+, P%
dbl (11): WC2, FT, QS, G, W, D, L, G+, G-, P+, P3+

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Country,WC1,WC2,FT,QS,G,W,D,L,G+,G-,GD,P2+,P%,P+,P3+
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
Germany,18,16,18,190,130,38,22,473,187,286,298:,82,"0,784",354,428
Brazil,20,20,20,196,127,41,28,420,161,259,295:,97,"0,753",349,422
Mexico,18,6,15,3,212,116,47,49,462,208,254,279:145,"0,658",340,395
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Montserrat,4,4,4,7,0,0,7,5,41,-36,0:,14,"0,000",0,0
North_Yemen,2,-6,2,8,0,0,8,1,17,-16,0:,16,"0,000",0,0
Brunei,2,-3,2,12,0,0,12,2,57,-55,0:,24,"0,000",0,0


## Preliminary exploratory data analysis


## Methods

## Expected outcomes and significance